Suppose we want to create the an integrand in QMCPy for evaluating the following integral: 
    $$ \int_{[0,1]^d} \|x\|_2^{\|x\|_2^{1/2}} dx,$$
where $[0,1]^d$ is the unit hypercube in $\mathbb{R}^d$. The integrand is defined everywhere except at $x=0$ and hence the definite integal is also defined.
    
    
The key in defining a Python function of an integrand in the QMCPy framework is that not only  it should be able to take one point $x \in \mathbb{R}^d$ and return a real value, but also that it would be able to take a set of $n$ sampling points as rows in a Numpy array of size $n \times d$ and return an array with $n$ values evaluated at each sampling point. The following examples illustrate this point.

In [1]:
from qmcpy import *
from numpy.linalg import norm as norm
from numpy import sqrt, array

Our first attempt maybe to create the integrand as a Python function as follows:

In [2]:
def f(x): return norm(x) ** sqrt(norm(x))

It looks reasonable except that maybe the Numpy function norm is executed twice. It's okay for now. Let us quickly test if the function behaves as expected at a point value:

In [3]:
x = 0.01
f(x)

0.6309573444801932

What about an array that represents $n=3$ sampling points in a two-dimensional domain, i.e., $d=2$? 

In [4]:
x = array([[1, 0], 
           [0, 0.01],
           [0.04, 0.04]])

In [5]:
f(x)

1.001650000560437

Now, the function should have returned $n=3$ real values that correponding to each of the sampling points. Let's debug our Python function.

In [6]:
norm(x)

1.0016486409914407

Numpy's norm(x) is obviously a matrix norm, but we want it to be vector 2-norm that acts on each row of x. To that end, let's add an axis argument to the function:

In [7]:
norm(x, axis = 1)

array([1.        , 0.01      , 0.05656854])

Now it's working! Let's make sure that the sqrt function is acting on each element of the vector norm results:

In [8]:
sqrt(norm(x, axis = 1))

array([1.        , 0.1       , 0.23784142])

It is. Putting everything together, we have:

In [9]:
norm(x, axis = 1) ** sqrt(norm(x, axis = 1))

array([1.        , 0.63095734, 0.50502242])

We have got our proper function definition now.

In [10]:
def f(x): x_norms = norm(x, axis = 1); return x_norms ** sqrt(x_norms)

We can now create an integrand instance with our QuickConstruct class in QMCPy and then invoke QMCPy's integrate function:

In [11]:
integrand = QuickConstruct(custom_fun=f, dimension=1)
sol, data = integrate(integrand)
data.summarize()

Solution: 0.6563         
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
QuickConstruct (Integrand Object)
                dimension: 1
               custom_fun: def f(x): x_norms = norm(x, axis = 1); return x_norms ** sqrt(x_norms)
IIDStdUniform (Discrete Distribution Object)
Uniform (True Distribution Object)
                dimension: 1 
CLT (StoppingCriterion Object)
                  abs_tol: 0.0100         
                  rel_tol: 0.0000         
                    n_max: 100000000
                  inflate: 1.2000         
                    alpha: 0.0100        
MeanVarData (Data Object)
                        n: [3230.]        
                  n_total: 4254           
               time_total: 0.0015         
               confid_int: [0.64617159 0.66637741]



For our integral, we know the true value. Let's check if QMCPy's solution is accurate enough:

In [12]:
true_sol = 0.658582  # In WolframAlpha: Integral[x**Sqrt[x], {x,0,1}]
abs_tol = data.stopping_criterion.abs_tol
qmcpy_error = abs(true_sol - sol)
print(qmcpy_error < abs_tol)

True


It's good. Shall we test the function with $d=2$ by simply changing the input parameter value of dimension for QuickConstruct?

In [13]:
integrand2 = QuickConstruct(custom_fun=f, dimension=2)
sol2, data2 = integrate(integrand2)
data2.summarize()

Solution: 0.8328         
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
QuickConstruct (Integrand Object)
                dimension: 2
               custom_fun: def f(x): x_norms = norm(x, axis = 1); return x_norms ** sqrt(x_norms)
IIDStdUniform (Discrete Distribution Object)
Uniform (True Distribution Object)
                dimension: 2 
CLT (StoppingCriterion Object)
                  abs_tol: 0.0100         
                  rel_tol: 0.0000         
                    n_max: 100000000
                  inflate: 1.2000         
                    alpha: 0.0100        
MeanVarData (Data Object)
                        n: [5719.]        
                  n_total: 6743           
               time_total: 0.0016         
               confid_int: [0.82286733 0.8426921 ]



Once again, we could test for accuracy of QMCPy with respect to the true value:

In [14]:
true_sol2 = 0.827606  # In WolframAlpha: Integral[Sqrt[x**2+y**2])**Sqrt[Sqrt[x**2+y**2]], {x,0,1}, {y,0,1}]
abs_tol2 = data2.stopping_criterion.abs_tol
qmcpy_error2 = abs(true_sol2 - sol2)
print(qmcpy_error2 < abs_tol2)

True
